In [19]:
import openpyxl
import ast
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
pd.set_option('display.max_colwidth', None)

In [9]:
df_xslx = pd.read_excel("../data/articles_with_full_text_20251004_105527.xlsx")
df_xslx.to_csv("../data/exmpl_for_clustering.csv", index = False)


In [ ]:

df = pd.read_csv("../data/exmpl_for_clustering.csv")
docs = df["abstract"].fillna("").astype(str)
docs_list = docs[docs.str.strip() != ""].tolist()           
N = len(docs_list)
if N < 3:
    raise ValueError(f"Занадто мало документів (N={N}). Додай текстів ≥3–5.")


nn = max(2, min(15, N - 1))
umap_model = UMAP(
    n_neighbors=nn,
    n_components=2,
    metric="cosine",
    init="random",
    random_state=42,
)
mcs = max(2, min(10, max(2, N // 2)))   
ms  = max(1, min(5, N - 1))            
hdbscan_model = HDBSCAN(
    min_cluster_size=mcs,
    min_samples=ms,
    metric="euclidean",                 
    cluster_selection_method="eom",
    prediction_data=False               
)
vectorizer_model = CountVectorizer(stop_words="english")

# У НАС ГОТОВІ ЕМБЕДДИНГИ, ТОМУ ПОТРІБНО БУДЕ ПЕРЕПИСАТИ
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(
    embedding_model=None,
    umap_model=umap_model,
    hdbscan_model = hdbscan_model,
    verbose=True,
    vectorizer_model=vectorizer_model
)


topics, probs = topic_model.fit_transform(docs_list)

# 5) Ключові слова теми для кожної статті
def topic_keywords_list(t, top_n=10):
    # if t == -1:
        # return ["Other"]
    words = topic_model.get_topic(t) or []
    return [w for w, _ in words[:top_n]] if words else ["Other"]

# Якщо треба рядок, заміни на: ", ".join(topic_keywords_list(t))
topic_kw = [topic_keywords_list(t, top_n=10) for t in topics]

# 6) Повертаємо в df (зберігаючи первісні індекси)
df = df.loc[docs.index].copy()
df["topic_id"] = topics
df["topic_keywords"] = topic_kw

print(df["topic_keywords"])


2025-10-04 17:23:58,923 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 1/1 [00:00<00:00,  4.22it/s]
2025-10-04 17:24:02,157 - BERTopic - Embedding - Completed ✓
2025-10-04 17:24:02,158 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-10-04 17:24:02,207 - BERTopic - Dimensionality - Completed ✓
2025-10-04 17:24:02,228 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-04 17:24:02,279 - BERTopic - Cluster - Completed ✓
2025-10-04 17:24:02,298 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-04 17:24:02,318 - BERTopic - Representation - Completed ✓


0    [microgravity, bone, cell, space, cells, increased, including, stem, loss, osteolysis]
1    [microgravity, bone, cell, space, cells, increased, including, stem, loss, osteolysis]
2    [microgravity, bone, cell, space, cells, increased, including, stem, loss, osteolysis]
Name: topic_keywords, dtype: object
